<a href="https://colab.research.google.com/github/aimazin/udacity-msft-capstone/blob/main/updatedAutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import DataImport
from azure.ai.ml.data_transfer import FileSystem
from azure.ai.ml import automl, Input
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_default_credential()
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AZUREML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

data_import = DataImport(name="ethdata")
ml_client.data.import_data(data_import=data_import)

my_training_data_input  = Input(type=AssetTypes.MLTABLE,
                                path="azureml://datastores/workspaceblobstore/paths/Forcasting/ethdata")

forecasting_job = automl.forecasting(
    compute=my_compute_name,
    experiment_name='model-*',
    training_data=my_training_data_input,
    target_column_name="y",
    primary_metric="normalized_root_mean_squared_error",
    enable_model_explainability=True
)

forecasting_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True,
)

forecasting_job.set_training(
    blocked_training_algorithms=["ARIMA"],
    enable_onnx_compatible_models=True
)

returned_sweep_job = ml_client.jobs.create_or_update(forecasting_job)

file_model = Model(
    path="mlflow-model/model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="model-1"
)


returned_sweep_job = ml_client.create_or_update(file_model)


returned_sweep_job = ml_client.online_endpoints.invoke(
    endpoint_name='aut_eth',
    request_file="../model-1/sample-request.json",
    local=True,
)

endpoint = ml_client.online_endpoints.get('aut_eth', local=True)

endpoint = ml_client.create_or_update(endpoint, local=True)

scoring_uri = endpoint.scoring_uri

